In [17]:
from docplex.mp.model import Model
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gurobipy as grb

In [153]:
def solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param):
    mdl = Model("MILP_with_lambda")
    mdl.parameters.timelimit = 2 * 60

    d = P.shape[1]  
    num_P = len(P)
    num_N = len(N)

    x_vars = mdl.binary_var_list(num_P, name="x")
    y_vars = mdl.binary_var_list(num_N, name="y")
    w = mdl.continuous_var_list(d, name="w") 
    c = mdl.continuous_var(name="c")  
    V = mdl.continuous_var(name="V", lb=0) 

    mdl.add_constraint(V >= (theta - 1) * mdl.sum(x_vars) + theta * mdl.sum(y_vars) + theta * R)

    for i, s in enumerate(P):
        dot_product = mdl.sum(s[j] * w[j] for j in range(d))
        mdl.add_constraint(x_vars[i] <= 1 + dot_product - c - epsilon_P)

    for i, s in enumerate(N):
        dot_product = mdl.sum(s[j] * w[j] for j in range(d))
        mdl.add_constraint(y_vars[i] >= dot_product - c + epsilon_N)

    objective = mdl.sum(x_vars) - lambda_param * V
    mdl.maximize(objective)

    solution = mdl.solve()

    # mdl.solution.export("solution.json") 
    # mdl.write("lpex1.lp")
    nodes = mdl.solve_details.nb_nodes_processed



    if solution:
        x_values = [solution[x] for x in x_vars]
        y_values = [solution[y] for y in y_vars]
        w_values = [solution[w] for w in w]
        c_value = solution[c]
        V_value = solution[V]

        print("Solution found:")
        print("x values:", x_values)
        print("y values:", y_values)
        print("w values:", w_values)
        print("c value:", c_value)
        print("V value:", V_value)

        return solution, x_values, y_values, w_values, c_value, V_value, nodes
    else:
        print("No solution found.")
        return None


In [235]:
def solve_milp_with_lambda(P, N, theta, epsilon_R, epsilon_P, epsilon_N, lambda_param):
    mdl = grb.Model("MILP_with_lambda")
    mdl.setParam("TimeLimit", 2 * 60)

    d = P.shape[1] 
    num_P = len(P)
    num_N = len(N)

    x_vars = mdl.addVars(num_P, vtype=grb.GRB.BINARY, name="x")
    y_vars = mdl.addVars(num_N, vtype=grb.GRB.BINARY, name="y")
    w = mdl.addVars(d, vtype=grb.GRB.CONTINUOUS, name="w")  
    c = mdl.addVar(vtype=grb.GRB.CONTINUOUS, name="c")  
    V = mdl.addVar(vtype=grb.GRB.CONTINUOUS, lb=0, name="V")  

    mdl.addConstr(V >= (theta - 1) * grb.quicksum(x_vars[i] for i in range(num_P)) + theta * grb.quicksum(y_vars[i] for i in range(num_N)) + theta * epsilon_R)

    for i, s in enumerate(P):
        dot_product = grb.quicksum(s[j] * w[j] for j in range(d))
        mdl.addConstr(x_vars[i] <= 1 + dot_product - c - epsilon_P)

    for i, s in enumerate(N):
        dot_product = grb.quicksum(s[j] * w[j] for j in range(d))
        mdl.addConstr(y_vars[i] >= dot_product - c + epsilon_N)

    objective = grb.quicksum(x_vars[i] for i in range(num_P)) - lambda_param * V
    mdl.setObjective(objective, grb.GRB.MAXIMIZE)

    mdl.optimize()
    nodes = mdl.NodeCount

    if mdl.status == grb.GRB.OPTIMAL:
        # mdl.write('sol.lp')
        x_values = [x_vars[i].x for i in range(num_P)]
        y_values = [y_vars[i].x for i in range(num_N)]
        w_values = [w[i].x for i in range(d)]
        c_value = c.x
        V_value = V.x

        print("Solution found:")
        print("x values:", x_values)
        print("y values:", y_values)
        print("w values:", w_values)
        print("c value:", c_value)
        print("V value:", V_value)

        return mdl, x_values, y_values, w_values, c_value, V_value, nodes
    else:
        print("No optimal solution found.")
        return None

In [28]:
# theta0 = 99
# theta1 = 100
# theta = theta0 / theta1
R = 3e-3
epsilon_P = 1e-3
epsilon_N = 2e-3
# lambda_param = (len(P) + 1) * theta1

# Breast Cancer

In [20]:
theta0 = 99 
theta1 = 100
theta = theta0/theta1

In [21]:
from sklearn.datasets import load_breast_cancer

In [22]:
d = load_breast_cancer()

In [23]:
X,y = d['data'], d['target']

In [24]:
P = np.array([x for idx, x in enumerate(X) if y[idx] == 1])
N = np.array([x for idx, x in enumerate(X) if y[idx] == 0])

In [25]:
lambda_param = (len(P) + 1) * theta1

In [64]:
solution, x_values, y_values, w_values, c_value, V_value, nodes = solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param)

Solution found:
x values: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [36]:
mdl, x_values, y_values, w_values, c_value, V_value, nodes = solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param)

Set parameter TimeLimit to value 120
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Arch Linux")

CPU model: Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  120

Optimize a model with 570 rows, 601 columns and 18700 nonzeros
Model fingerprint: 0x3c069b2e
Variable types: 32 continuous, 569 integer (569 binary)
Coefficient statistics:
  Matrix range     [7e-04, 4e+03]
  Objective range  [1e+00, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-03, 1e+00]
Found heuristic solution: objective -106.3258260
Presolve time: 0.02s
Presolved: 570 rows, 601 columns, 18700 nonzeros
Variable types: 32 continuous, 569 integer (569 binary)

Root relaxation: objective 3.570000e+02, 272 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Inc

In [37]:
nodes

27759.0

In [38]:
x_tilde = np.array([0 if np.dot(w_values, s) + np.sum(c_value) < epsilon_P else 1 for s in P])


In [39]:
np.sum(x_tilde)

357

# Wine Quality Red

In [40]:
wq_red_df = pd.read_csv('./data/wine-quality/winequality-red.csv', delimiter=';')

In [41]:
wq_red_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [42]:
X, y = wq_red_df.drop(columns=['quality']).to_numpy() , wq_red_df['quality'].to_numpy()     

In [43]:
y = y >= 8

In [44]:
y = y.astype(int)

In [45]:
theta0 = 4
theta1 = 100
theta = theta0 / theta1

In [46]:
P = np.array([x for idx, x in enumerate(X) if y[idx] == 1])
N = np.array([x for idx, x in enumerate(X) if y[idx] == 0])

In [47]:
lambda_param = (len(P) + 1) * theta1

In [78]:
solution, x_values, y_values, w_values, c_value, V_value, nodes = solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param)

Solution found:
x values: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
y values: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 1.0, 1.0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 1.0, 0, 1.0, 0, 0, 1.0, 1.0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 1.0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 1.0, 1.0, 0, 0, 0, 1.0, 1.0, 0, 0, 1.0, 1.0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 0, 0, 0, 1.0, 0

In [48]:
mdl, x_values, y_values, w_values, c_value, V_value, nodes = solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param)

Set parameter TimeLimit to value 120
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Arch Linux")

CPU model: Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  120

Optimize a model with 1600 rows, 1612 columns and 22255 nonzeros
Model fingerprint: 0xe6a6e480
Variable types: 13 continuous, 1599 integer (1599 binary)
Coefficient statistics:
  Matrix range     [1e-02, 3e+02]
  Objective range  [1e+00, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-04, 1e+00]
Found heuristic solution: objective -0.2279982
Presolve time: 0.02s
Presolved: 1600 rows, 1612 columns, 22255 nonzeros
Variable types: 13 continuous, 1599 integer (1599 binary)

Root relaxation: objective 1.800000e+01, 1587 iterations, 0.08 seconds (0.10 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntIn

In [49]:
nodes

503.0

In [50]:
x_tilde = np.array([0 if np.dot(w_values, s) + np.sum(c_value) < epsilon_P else 1 for s in P])


In [51]:
np.sum(x_tilde)

18

# Wine Quality White

In [52]:
wq_white_df = pd.read_csv('./data/wine-quality/winequality-white.csv', delimiter=';')

In [53]:
wq_white_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [54]:
X, y = wq_white_df.drop(columns=['quality']).to_numpy() , wq_white_df['quality'].to_numpy()     

In [55]:
y = y >= 8

In [56]:
y = y.astype(int)

In [57]:
theta0 = 10
theta1 = 100
theta = theta0 / theta1

In [58]:
P = np.array([x for idx, x in enumerate(X) if y[idx] == 1])
N = np.array([x for idx, x in enumerate(X) if y[idx] == 0])

In [59]:
lambda_param = (len(P) + 1) * theta1

In [124]:
solution, x_values, y_values, w_values, c_value, V_value, nodes = solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param)

Solution found:
x values: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
y values: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [60]:
mdl, x_values, y_values, w_values, c_value, V_value, nodes = solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param)

Set parameter TimeLimit to value 120
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Arch Linux")

CPU model: Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  120



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [128]:
nodes

4705

In [162]:
x_tilde = np.array([0 if np.dot(w_values, s) + np.sum(c_value) < epsilon_P else 1 for s in P])


In [164]:
np.sum(x_tilde)

180

# South German Credit

In [61]:
cols_english = ["running account", "term", "morality", "expenditure", "amount", "savings account", "possession", "installment", "family status", "guarantor", "length of residence", "assets", "age", "long-term credit", "residence", "previous credit", "occupation", "personal", "telephone", "guest worker", "credit"]

In [224]:
south_german_credit_df = pd.read_csv('./data/south-german-credit/SouthGermanCredit.dat', skiprows=1, header=None, delimiter=' ')

In [225]:
south_german_credit_df.head()

,0,1,2,3,4,5,6
0,36.0,2384.0,2.0,4.0,1.0,33.0,1.0
1,18.0,1216.0,2.0,4.0,3.0,23.0,1.0
2,18.0,1864.0,3.0,4.0,2.0,30.0,2.0
3,36.0,4455.0,3.0,2.0,2.0,30.0,2.0
4,15.0,3959.0,3.0,3.0,2.0,29.0,1.0


In [226]:
X, y = south_german_credit_df.drop(columns=[6]).to_numpy() , south_german_credit_df[6].to_numpy()     

In [227]:
len(X)

1000

In [228]:
theta0 = 90
theta1 = 100
theta = theta0 / theta1

In [229]:
P = np.array([x for idx, x in enumerate(X) if y[idx] == 1])
N = np.array([x for idx, x in enumerate(X) if y[idx] == 0])

In [230]:
lambda_param = (len(P) + 1) * theta1

In [140]:
solution, x_values, y_values, w_values, c_value, V_value, nodes = solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param)

Solution found:
x values: [0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 0, 0, 0, 0, 1.0, 0, 1.0, 1.0, 1.0, 0, 1.0, 0, 0, 0, 1.0, 0, 0, 1.0, 1.0, 0, 0, 1.0, 0, 0, 0, 1.0, 0, 1.0, 0, 0, 1.0, 0, 0, 0, 1.0, 1.0, 1.0, 0, 0, 0, 0, 1.0, 1.0, 0, 0, 1.0, 1.0, 0, 1.0, 1.0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 0, 0, 1.0, 0, 0, 0, 1.0, 1.0, 1.0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 1.0, 0, 1.0, 0, 1.0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 1.0, 0, 1.0, 0, 0, 0, 0, 1.0, 0, 0, 0, 1.0, 0, 0, 1.0, 0, 1.0, 0, 1.0, 1.0, 0, 1.0, 0, 1.0, 0, 1.0, 0, 1.0, 0, 0, 0, 0, 0, 1.0, 0, 0, 1.0, 0, 0, 0, 1.0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 1.0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [236]:
mdl, x_values, y_values, w_values, c_value, V_value, nodes = solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param)

Set parameter TimeLimit to value 120
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Arch Linux")

CPU model: Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  120

Optimize a model with 634 rows, 641 columns and 5698 nonzeros
Model fingerprint: 0x96002003
Variable types: 8 continuous, 633 integer (633 binary)
Coefficient statistics:
  Matrix range     [1e-01, 2e+04]
  Objective range  [1e+00, 6e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 1e+00]
Found heuristic solution: objective 633.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 633 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.330000000000e+02, best bound 6.330000000000e+02, gap 0.0000%
Solution found:
x values: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 

In [237]:
nodes

0.0

In [238]:
x_tilde = np.array([0 if np.dot(w_values, s) + np.sum(c_value) <= epsilon_P  else 1 for s in P])


In [239]:
np.sum(x_tilde)

633

# Crop Mapping Dataset

In [75]:
crop_df = pd.read_csv('./data/crops_new/WinnipegDataset.txt', delimiter=',')

In [76]:
crop_df.head()

,label,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174
0,1,-13.559,-21.407,-11.4040,-15.248,-11.923,-15.291,-2.1548,-7.8474,-10.0020,...,0.18519,0.72602,5.3333,6.000000,0.29489,9.77780,2.44440,1.67700,0.20988,0.65422
1,1,-12.802,-20.335,-10.3990,-14.132,-11.096,-14.361,-2.4039,-7.5330,-9.9369,...,0.33333,-0.48751,2.1111,0.098765,0.83333,0.33333,0.33333,0.84869,0.50617,-0.18898
2,1,-12.431,-19.902,-10.0740,-13.598,-10.829,-14.048,-2.3566,-7.4717,-9.8283,...,0.25926,0.25298,2.2222,0.172840,0.68889,0.88889,0.66667,1.27300,0.30864,0.10483
3,1,-12.689,-19.529,-10.0280,-13.350,-11.056,-14.014,-2.6611,-6.8396,-9.5006,...,0.16049,0.43750,4.1111,0.320990,0.83333,0.33333,0.33333,1.14910,0.38272,0.41603
4,1,-12.686,-19.278,-9.8185,-13.108,-10.932,-13.939,-2.8675,-6.5919,-9.4594,...,0.18519,0.35000,4.0000,0.444440,0.68889,0.88889,0.66667,1.58110,0.20988,0.50000


In [77]:
X, y = crop_df.drop(columns=['label']).to_numpy() , crop_df['label'].to_numpy()     

In [78]:
idx = np.random.choice(np.arange(len(X)), int(len(X)/10), replace=False)
# 

In [79]:
X = X[idx]
y = y[idx]

In [80]:
y = (y == 6).astype(int)

In [81]:
theta0 = 99
theta1 = 100
theta = theta0 / theta1

In [82]:
P = np.array([x for idx, x in enumerate(X) if y[idx] == 1])
N = np.array([x for idx, x in enumerate(X) if y[idx] == 0])

In [83]:
lambda_param = (len(P) + 1) * theta1

In [13]:
solution, x_values, y_values, w_values, c_value, V_value, nodes = solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param)

Solution found:
x values: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [84]:
mdl, x_values, y_values, w_values, c_value, V_value, nodes = solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param)

Set parameter TimeLimit to value 120
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Arch Linux")

CPU model: Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  120



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [ ]:
nodes

In [14]:
x_tilde = np.array([0 if np.dot(w_values, s) + np.sum(c_value) <= epsilon_P  else 1 for s in P])


In [15]:
np.sum(x_tilde)

0

# Synthetic Datasets

In [201]:
from synthetic_datasets import ClusterDataset

In [202]:
cd = ClusterDataset()

In [203]:
X, y = cd.generate()

In [204]:
theta0 = 99 
theta1 = 100
theta = theta0/theta1

In [205]:
P = np.array([x for idx, x in enumerate(X) if y[idx] == 1])
N = np.array([x for idx, x in enumerate(X) if y[idx] == 0])

In [206]:
lambda_param = (len(P) + 1) * theta1

In [207]:
solution, x_values, y_values, w_values, c_value, V_value, nodes = solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param)

Solution found:
x values: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 0, 0, 0]
y values: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [208]:
# mdl, x_values, y_values, w_values, c_value, V_value, nodes = solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param)

In [209]:
nodes

426

In [210]:
x_tilde = np.array([0 if np.dot(w_values, s) + np.sum(c_value) < epsilon_P else 1 for s in P])


In [211]:
np.sum(x_tilde)

80

In [190]:
from synthetic_datasets import TwoClusterDataset

In [191]:
cd = TwoClusterDataset()

In [192]:
X, y = cd.generate()

In [193]:
theta0 = 99 
theta1 = 100
theta = theta0/theta1

In [194]:
P = np.array([x for idx, x in enumerate(X) if y[idx] == 1])
N = np.array([x for idx, x in enumerate(X) if y[idx] == 0])

In [195]:
lambda_param = (len(P) + 1) * theta1

In [196]:
solution, x_values, y_values, w_values, c_value, V_value, nodes = solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param)

Solution found:
x values: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0]
y values: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [197]:
# mdl, x_values, y_values, w_values, c_value, V_value, nodes = solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param)

In [198]:
nodes

315

In [199]:
x_tilde = np.array([0 if np.dot(w_values, s) + np.sum(c_value) < epsilon_P else 1 for s in P])


In [200]:
np.sum(x_tilde)

80

In [177]:
from synthetic_datasets import DiffusedBenchmark

In [178]:
cd = DiffusedBenchmark()

In [179]:
X, y = cd.generate()

In [180]:
theta0 = 99 
theta1 = 100
theta = theta0/theta1

In [181]:
P = np.array([x for idx, x in enumerate(X) if y[idx] == 1])
N = np.array([x for idx, x in enumerate(X) if y[idx] == 0])

In [182]:
lambda_param = (len(P) + 1) * theta1

In [183]:
solution, x_values, y_values, w_values, c_value, V_value, nodes = solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param)

Solution found:
x values: [1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 1.0, 0, 0, 1.0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 1.0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
y values: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [184]:
# mdl, x_values, y_values, w_values, c_value, V_value, nodes = solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param)

In [188]:
nodes

26094

In [189]:
x_tilde = np.array([0 if np.dot(w_values, s) + np.sum(c_value) < epsilon_P else 1 for s in P])


In [187]:
np.sum(x_tilde)

200

In [165]:
from synthetic_datasets import PrismDataset

In [166]:
cd = PrismDataset()

self.d0 2


In [167]:
X, y, _ = cd.generate()

In [168]:
theta0 = 99 
theta1 = 100
theta = theta0/theta1

In [169]:
P = np.array([x for idx, x in enumerate(X) if y[idx] == 1])
N = np.array([x for idx, x in enumerate(X) if y[idx] == 0])

In [170]:
lambda_param = (len(P) + 1) * theta1

In [171]:
solution, x_values, y_values, w_values, c_value, V_value, nodes = solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param)

Solution found:
x values: [0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0]
y values: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [172]:
# mdl, x_values, y_values, w_values, c_value, V_value, nodes = solve_milp_with_lambda(P, N, theta, R, epsilon_P, epsilon_N, lambda_param)

In [173]:
nodes

0

In [174]:
x_tilde = np.array([0 if np.dot(w_values, s) + np.sum(c_value) < epsilon_P else 1 for s in P])


In [175]:
np.sum(x_tilde)

12